# Skywalker algorithm against the two mirror system

In [1]:
SIMULATION = False

goal1 = 220 # Target pixel on first imager. NOTE: Don't do the math for flipped image
goal2 = 250 # Target pixel on second imager. NOTE: Don't do the math for flipped image
first_steps = -10 # Naive first step for the parameters search
tolerances = 20.0 # Tolerance of each target in Pixels
average = 50 # Number of shots to average over
timeout = 600 # Timeout in seconds
log_level = "INFO"

# PARAMETERS FOR SIMULATION ONLY
centroid_noise = 5.0 # Noise level of centroid measurements
infinite_yag = False # Assume all yags are infinitely large

goals = [goal1, goal2]

In [2]:
############
# Standard #
############
import os
import sys
import logging
import argparse

###############
# Third Party #
###############
from bluesky import RunEngine
from bluesky.tests.utils import MsgCollector
from bluesky.plans import run_wrapper
from pcdsdevices.sim import pim, source, mirror

###############
#  pswalker   #
###############
from pswalker.examples  import patch_pims
from pswalker.skywalker import skywalker
from pswalker.skywalker import homs_RE
from pswalker.config import homs_system

/reg/g/pcds/pyps/conda/rhel6/envs/skywalker/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
"""
Parameters
----------
"""
#Configure logger
log_level = getattr(logging, log_level, None)

#Report invalid logging level
if not isinstance(log_level, int):
    raise ValueError("Invalid log level : {}".format(log_level))

#Create basic configuration
logging.basicConfig(level=log_level,
                format='%(asctime)s - %(levelname)s - %(message)s')

#Instantiate simulation
if SIMULATION:
    s = source.Undulator('test_undulator')
    m1 = mirror.OffsetMirror('test_m1h', z=90.510, alpha=0.0014)
    m2 = mirror.OffsetMirror('test_m2h', x=0.0317324, z=101.843, alpha=0.0014)
    y1 = pim.PIM('test_p3h', x=0.0317324, z=103.660,
             zero_outside_yag= not infinite_yag)
    y2 = pim.PIM('test_dg3', x=0.0317324, z=375.000,
             zero_outside_yag= not infinite_yag)
    patch_pims([y1, y2], mirrors=[m1, m2], source=s)

    #Add noise
    y1.centroid_noise = centroid_noise
    y2.centroid_noise = centroid_noise    
else:
    print("*"*80)
    print("* WARNING: Running on real devices in Production...")
    print("*"*80)
    input("Press any key to continue...")
    system = homs_system()
    m1h = system['m1h']
    m1h2 = system['m1h2']
    m2h = system['m2h']
    m2h2 = system['m2h2']
    xrtm2 = system['xrtm2']
    xrtm22 = system['xrtm22']
    hx2 = system['hx2']
    dg3 = system['dg3']
    mfxdg1 = system['mfxdg1']
    m1 = m1h
    m2 = m2h
    y1 = hx2
    y2 = dg3


********************************************************************************
* WARNING: Running on real devices in Production...
********************************************************************************
Press any key to continue...


In [4]:
#Create Skywalker plan
# NOTE : the image is rotated 90 degrees, this means we need to be aligning to centroid_y
plan = skywalker([y1, y2], [m1, m2], 'detector_stats2_centroid_y', 'pitch',
             goals, first_steps=first_steps, tolerances=tolerances,
             averages=average, timeout=timeout)


In [5]:
#Create RunEngine
RE = None
if SIMULATION:
    RE = RunEngine({})
    collector = MsgCollector()
    RE.msg_hook = collector
else:
    RE = homs_RE()

In [6]:
RE(plan))

IllegalMessageSequence: A 'close_run' message was not received before the 'open_run' message

In [ ]:
if SIMULATION:
    #Analyze Performance
    print(len(RE.msg_hook.msgs))